In [1]:
from omegaconf import OmegaConf
import sentencepiece
from tqdm import tqdm

In [2]:
conf = {'processed_path': "./processed_data", "split": "train", "context_length": 512, 'stride_big_sequences': 256, "count_past_boards": 2, "in_memory":False, 'target_types': [0, 1, 2, 3, 4]}
oconf = OmegaConf.create(conf)

In [3]:
sp = sentencepiece.SentencePieceProcessor("./artifacts/old/artifacts_old/artifacts/sp1000.model")

In [4]:
from data.AlphazeroCommentaryDataset import AlphazeroCommentaryDataset

In [5]:
ds = AlphazeroCommentaryDataset(oconf, sp)

In [6]:
from torch.nn.utils.rnn import pad_sequence
import torch
from torch.utils.data import DataLoader

def collate_fn(data):
    board_data = torch.stack(list(map(lambda x: x[0], data)))
    sequences = pad_sequence(list(map(lambda x: x[1], data)), batch_first=True,
                             padding_value=sp.pad_id())
    types = torch.stack(list(map(lambda x: x[2], data)))
    X_sequence = sequences[:, :-1]
    y_sequence = sequences[:, 1:]
    next_pad_mask = (y_sequence == sp.pad_id())
    return board_data, X_sequence, y_sequence, next_pad_mask, types

dl = DataLoader(
            ds,
            batch_size=128,
            collate_fn=collate_fn,
            num_workers=2
        )

In [ ]:
import wandb
run = wandb.init()
artifact = run.use_artifact('georgerapeanu/thesis/model_val_loss:v68', type='model')
artifact_dir = artifact.download()

In [7]:
model = torch.load("./artifacts/model_val_loss:v68/model_save_val_loss_2.341321006943198", map_location=torch.device("cpu"))
model.eval()

Model(
  (board_preparation): Sequential(
    (0): Conv2d(125, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (emb): Embedding(1000, 256)
  (pe_board): PositionalEncoding2D(
    (Dropout): Dropout(p=0.1, inplace=False)
  )
  (pe_text): PositionalEncoding1D(
    (Dropout): Dropout(p=0.1, inplace=False)
  )
  (encoders): ModuleList(
    (0-5): 6 x TransformerEncoderBlock(
      (mha): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
      )
      (layer_norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (feed_forward): Sequential(
        (0): Linear(in_features=256, out_features=512, bias=True)
        (1): ReLU(inplace=True)
        (2): Linear(in_features=512, out_features=256, bias=True)
      )
      (layer_norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
   

In [8]:
import random

choices = random.sample(range(len(ds)), 10)
to_predict = [ds[i] for i in choices]
to_predict_metadata = [ds.get_raw_data(i) for i in choices]

In [9]:
from model.predict import Predictor
predictor = Predictor(512, sp)

In [10]:
import chess

for ((X_board, y_tokens, _), (current_board, past_board, current_eval, past_eval)) in zip(to_predict,
                                                                                                  to_predict_metadata):
    predicted_text = predictor.predict(model, X_board, '', 1024)
    actual_text = predictor.tokens_to_string(y_tokens)
    print(chess.Board(past_board))
    print(chess.Board(current_board))
    print("actual:" + actual_text)
    print("predicted:" + predicted_text)

r . b q k b . r
p p p p . . . p
. . n . . n N .
. . . . . p . .
. . . . P . . .
. . . . . . . Q
P P P P . P P P
R N B . K B . R
r . b q k b r .
p p p p . . . p
. . n . . n N .
. . . . . p . .
. . . . P . . .
. . . . . . . Q
P P P P . P P P
R N B . K B . R
actual:The question of relocation is put to the knight.
predicted:black knight moves to g3 to attack the bishop and proceed to retreat the bishop
r . b . . r k .
p p . . . . p p
. . n q p . . .
. . . p . . . .
. . . P . N n .
. . . B . N P .
P P . Q . P . P
R . . . . R K .
r . . . . r k .
p p . b . . p p
. . n q p . . .
. . . p . . . .
. . . P . N n .
. . . B . N P .
P P . Q . P . P
R . . . . R K .
actual:GK GameDB suggests 15 .. e5 , but after 16. dxe5 Nxe5 17. Nxe5 Nxe5 etc. I've got an isolated pawn and no piece attack, so I prefer to wait and try to "develop" the Bishop in another way.
predicted:At this point, I'm starting to play e6 normally.
N . b . r . . .
p p . k . p . .
. . n p . . . p
. B . . b . p .
. . . p P . n .
. . . . 

In [10]:
optimizer = (torch.optim.SGD(model.parameters(), lr=0.08))

In [11]:
model.train()
train_losses = []
for _ in range(2):
    train_losses.clear() 
    for i, batch in enumerate(tqdm(dl)):
        if i > 6:
            break
        optimizer.zero_grad()
        (X_board, X_text, y_sequence, pad_mask, types) = batch
        _, loss = model(X_board, X_text, pad_mask, y_sequence)
        train_losses.append(loss.item())
        loss.backward()
        optimizer.step()
    
    train_loss = sum(train_losses) / len(train_losses)
    print(train_loss)

  1%|          | 7/920 [00:47<1:43:31,  6.80s/it]


0.6018177483763013


  1%|          | 7/920 [00:44<1:36:19,  6.33s/it]

0.5914961780820575


In [13]:
# tworuns of eval to see they match
print(train_losses)

[0.2941211462020874, 0.4395594298839569, 0.7215953469276428, 0.5389072895050049, 0.7495558857917786, 0.962472140789032, 0.3383268713951111, 0.4746575355529785, 0.6347800493240356, 0.3580690920352936, 0.42043012380599976]


In [13]:
print(train_losses)

[0.2941211462020874, 0.4395594298839569, 0.7215953469276428, 0.5389072895050049, 0.7495558857917786, 0.962472140789032, 0.3383268713951111, 0.4746575355529785, 0.6347800493240356, 0.3580690920352936, 0.42043012380599976]


In [12]:
# two train runs, 10 vs 20 epochs
print(train_losses)

[0.30783262848854065, 0.46060094237327576, 0.7480670809745789, 0.5648900270462036, 0.7838973999023438, 1.0042369365692139, 0.3544386625289917, 0.49887675046920776, 0.6658899188041687, 0.3742157518863678, 0.43792301416397095]


In [12]:
print(train_losses)

[0.3074360191822052, 0.4583767354488373, 0.7452166080474854, 0.5626044869422913, 0.7766403555870056, 0.9388211369514465, 0.3513779044151306]


In [1]:
1+1

2